In [83]:
import pandas as pd
import  math
import os
import bson
from dateutil import parser

from pymongo import MongoClient

def mongoCollection(connstr, db, collection):
    client = MongoClient(connstr)
    db = client[db]
    return db[collection]

In [59]:
collection = mongoCollection(os.environ.get('MONGODB_CONN_STR'), 'smsinfo', 'transactions')

In [108]:
csvdf = pd.read_csv('/data/2020-04-02T10-51-33.741379-csv-updated.csv')

In [109]:
csvdf

,_id,transaction.expense_amount,transaction.payment_mode,transaction.merchant,transaction.datetime,transaction.category,transaction.sub_category,transaction.comment,transaction.direction,transaction.service_provider,transaction.available_balance,transaction.outstanding_amount,event.name
0,5e6344530b2e08ee3d72852e,270.0,Akshayakalpa Wallet,Akshayakalpa,2020-03-07 14:10:11.095,grocery,diary,NaN,DEBIT,NaN,260.0,NaN,NaN
1,5e63a8b4a2528f380776e94d,140.0,Akshayakalpa Wallet,Akshayakalpa,2020-03-04 14:10:11.099,grocery,diary,NaN,DEBIT,NaN,120.0,NaN,NaN
2,5e6a2e2fcf5bc03d3f440ca7,270.0,Akshayakalpa Wallet,Akshayakalpa,2020-03-12 13:10:11.142,grocery,diary,NaN,DEBIT,NaN,350.0,NaN,NaN
3,5e74e355b03788edf71b9989,270.0,Akshayakalpa Wallet,Akshayakalpa,2020-03-20 15:48:37.777,grocery,diary,NaN,DEBIT,NaN,80.0,NaN,NaN
4,NaN,400.0,Akshayakalpa Wallet,Akshayakalpa,2020-03-22 16:52:06.466,grocery,diary,NaN,DEBIT,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,5e6a2e2fcf5bc03d3f440cae,600.0,XXX983,paytmqr2810050501011f9xdbj3suxl@paytm,2020-03-11 00:00:00.000,medicine,medicine,NaN,DEBIT,NaN,NaN,NaN,NaN
81,5e79cdb421664b08c327c20b,37.0,XXX983,paytmqr2810050501011f9xdbj3suxl@paytm,2020-03-23 00:00:00.000,medicine,medicine,NaN,DEBIT,NaN,NaN,NaN,NaN
82,5e858a11e0428e02c2b4e999,550.0,XXX983,paytmqr2810050501011f9xdbj3suxl@paytm,2020-03-23 00:00:00.000,medicine,medicine,NaN,DEBIT,NaN,NaN,NaN,NaN
83,5e85c113e0428e02c2b4e99e,550.0,XXX983,paytmqr2810050501011f9xdbj3suxl@paytm,2020-03-23 00:00:00.000,medicine,medicine,NaN,DEBIT,NaN,NaN,NaN,NaN


In [110]:
keys = list(csvdf.to_dict().keys())
count = len(csvdf)

In [111]:
def dict_from_idx(dataframe, idx, keys):
    data = {}
    for k in keys:
        data[k] = dataframe[k][idx]
    return data

In [112]:
def prepare_dict_for_update(d={}):
    date = d.pop('transaction.datetime')
    _id = d.pop('_id')
    if type(_id) != str and math.isnan(float(_id)):
        d.update({'transaction.datetime':  parser.parse(date)})
        d.update({'status': {'analysis_done': True, 'mode': 'MANUAL'}})
        for key in list(d.keys()):
            _keys = key.split('.')
            if len(_keys) == 2:
                val = d.pop(key)
                if _keys[0] not in d:
                    d[_keys[0]] = {_keys[1] : val}
                else:
                    d[_keys[0]][_keys[1]] = val      
        return None, d
    else:
        return _id, d

In [113]:
for i in range(count):
    d = dict_from_idx(csvdf, i, keys)
    _id, d = prepare_dict_for_update(d=d)
    if _id == None:
        _id = collection.insert_one(d).inserted_id
    else:
        collection.update_one({'_id': bson.ObjectId(_id)}, {'$set': d}, upsert = False)
    print(_id)
    print(d)
        

5e6344530b2e08ee3d72852e
{'transaction.expense_amount': 270.0, 'transaction.payment_mode': 'Akshayakalpa Wallet', 'transaction.merchant': 'Akshayakalpa', 'transaction.category': 'grocery', 'transaction.sub_category': 'diary', 'transaction.comment': nan, 'transaction.direction': 'DEBIT', 'transaction.service_provider': nan, 'transaction.available_balance': 260.0, 'transaction.outstanding_amount': nan, 'event.name': nan}
5e63a8b4a2528f380776e94d
{'transaction.expense_amount': 140.0, 'transaction.payment_mode': 'Akshayakalpa Wallet', 'transaction.merchant': 'Akshayakalpa', 'transaction.category': 'grocery', 'transaction.sub_category': 'diary', 'transaction.comment': nan, 'transaction.direction': 'DEBIT', 'transaction.service_provider': nan, 'transaction.available_balance': 120.0, 'transaction.outstanding_amount': nan, 'event.name': nan}
5e6a2e2fcf5bc03d3f440ca7
{'transaction.expense_amount': 270.0, 'transaction.payment_mode': 'Akshayakalpa Wallet', 'transaction.merchant': 'Akshayakalpa', 

5e6344540b2e08ee3d728544
{'transaction.expense_amount': 77.0, 'transaction.payment_mode': 'CREDIT Card xx3690', 'transaction.merchant': 'SRI UDUPI PARK', 'transaction.category': 'food', 'transaction.sub_category': 'eating-out', 'transaction.comment': nan, 'transaction.direction': 'DEBIT', 'transaction.service_provider': nan, 'transaction.available_balance': 270186.0, 'transaction.outstanding_amount': 29814.0, 'event.name': nan}
5e6344540b2e08ee3d728545
{'transaction.expense_amount': 1249.62, 'transaction.payment_mode': 'CREDIT Card xx3690', 'transaction.merchant': 'BBPSBILLPAY', 'transaction.category': 'utility', 'transaction.sub_category': 'internet', 'transaction.comment': nan, 'transaction.direction': 'DEBIT', 'transaction.service_provider': 'Act', 'transaction.available_balance': 268937.16, 'transaction.outstanding_amount': 31062.84, 'event.name': nan}
5e63a8b4a2528f380776e94a
{'transaction.expense_amount': 410.0, 'transaction.payment_mode': 'CREDIT Card xx3690', 'transaction.merch

5e74e354b03788edf71b9980
{'transaction.expense_amount': 399.0, 'transaction.payment_mode': 'CREDIT Card xx3690', 'transaction.merchant': 'Bharat Sanchar Nigam L', 'transaction.category': 'utility', 'transaction.sub_category': 'Mobile', 'transaction.comment': nan, 'transaction.direction': 'DEBIT', 'transaction.service_provider': 'Bsnl-0703', 'transaction.available_balance': 252170.0, 'transaction.outstanding_amount': 47830.0, 'event.name': nan}
5e74e354b03788edf71b9982
{'transaction.expense_amount': 1313.56, 'transaction.payment_mode': 'CREDIT Card xx3690', 'transaction.merchant': 'www.bigbasket.', 'transaction.category': 'grocery', 'transaction.sub_category': 'all', 'transaction.comment': nan, 'transaction.direction': 'DEBIT', 'transaction.service_provider': nan, 'transaction.available_balance': 250857.3, 'transaction.outstanding_amount': 49142.7, 'event.name': nan}
5e74e355b03788edf71b9986
{'transaction.expense_amount': 377.26, 'transaction.payment_mode': 'CREDIT Card xx3690', 'transa

5e80d16248a45dfa05b4e70c
{'transaction.expense_amount': 290.0, 'transaction.payment_mode': 'CREDIT Card xx3690', 'transaction.merchant': 'Innovative Retail Concept', 'transaction.category': 'grocery', 'transaction.sub_category': 'all', 'transaction.comment': nan, 'transaction.direction': 'DEBIT', 'transaction.service_provider': nan, 'transaction.available_balance': 275634.01, 'transaction.outstanding_amount': 24365.99, 'event.name': nan}
5e80d16248a45dfa05b4e710
{'transaction.expense_amount': 546.0, 'transaction.payment_mode': 'CREDIT Card xx3690', 'transaction.merchant': 'SWIGGY95116', 'transaction.category': 'food', 'transaction.sub_category': 'Delivery', 'transaction.comment': nan, 'transaction.direction': 'DEBIT', 'transaction.service_provider': 'Punjabi-Rasoi', 'transaction.available_balance': 275267.0, 'transaction.outstanding_amount': 24733.0, 'event.name': nan}
5e80d16348a45dfa05b4e716
{'transaction.expense_amount': 257.0, 'transaction.payment_mode': 'CREDIT Card xx3690', 'tran

5e85c113e0428e02c2b4e99e
{'transaction.expense_amount': 550.0, 'transaction.payment_mode': 'XXX983', 'transaction.merchant': 'paytmqr2810050501011f9xdbj3suxl@paytm', 'transaction.category': 'medicine', 'transaction.sub_category': 'medicine', 'transaction.comment': nan, 'transaction.direction': 'DEBIT', 'transaction.service_provider': nan, 'transaction.available_balance': nan, 'transaction.outstanding_amount': nan, 'event.name': nan}
5e80d16048a45dfa05b4e6fa
{'transaction.expense_amount': 157.0, 'transaction.payment_mode': 'XXX983', 'transaction.merchant': 'paytmqr2810050501011f9xdbj3suxl@paytm', 'transaction.category': 'Medicine', 'transaction.sub_category': 'Medicine', 'transaction.comment': nan, 'transaction.direction': 'DEBIT', 'transaction.service_provider': nan, 'transaction.available_balance': nan, 'transaction.outstanding_amount': nan, 'event.name': nan}
